In [13]:
import tensorflow as tf
import numpy as np

# 数据集
input_data = ["2022年3月22日", "2021年7月4日"]
output_data = ["2022-3-22", "2021-7-4"]

# 建立词表
input_vocab = set(("".join(input_data)).split())
output_vocab = set(("".join(output_data)).split())

id2word_input = list(input_vocab)
input2id = {c: i for i, c in enumerate(id2word_input)}

id2word_output = list(output_vocab)
output2id = {c: i for i, c in enumerate(id2word_output)}

# 超参数
hidden_size = 128
embedding_size = 64
batch_size = 64
epochs = 100

# 构建编码器
def make_encoder(inputs, hidden_size, embedding_size):
    embedding = tf.Variable(tf.random_uniform([len(input_vocab), embedding_size], -1.0, 1.0))
    inputs = tf.nn.embedding_lookup(embedding, inputs)
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    _, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    return final_state

# 构建解码器
def make_decoder(inputs, hidden_size, embedding_size, output_length, initial_state):
    embedding = tf.Variable(tf.random_uniform([len(output_vocab), embedding_size], -1.0, 1.0))
    inputs = tf.nn.embedding_lookup(embedding, inputs)
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    outputs, _ = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
    dense_layer = tf.layers.Dense(len(output_vocab))
    logits = dense_layer(outputs)
    predictions = tf.argmax(logits, axis=2)
    return logits, predictions

# 构建图
graph = tf.Graph()
with graph.as_default():
    # 输入数据
    inputs = tf.placeholder(tf.int32, [batch_size, None])
    outputs = tf.placeholder(tf.int32, [batch_size, None])
    output_length = tf.placeholder(tf.int32, [batch_size])
    # 编码器
    encoder_state = make_encoder(inputs, hidden_size, embedding_size)
    # 解码器
    decoder_inputs = tf.pad(outputs[:, :-1], [[0, 0], [1, 0]], constant_values=len(output_vocab) - 1)
    decoder_initial_state = encoder_state
    decoder_logits, decoder_predictions = make_decoder(decoder_inputs, hidden_size, embedding_size, output_length, decoder_initial_state)
    # 损失函数
    mask = tf.sequence_mask(output_length, dtype=tf.float32)
    loss = tf.contrib.seq2seq.sequence_loss(decoder_logits, outputs, mask)
    # 优化器
    optimizer = tf.train.AdamOptimizer().minimize(loss)

# 训练模型
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        epoch_loss = 0
        for i in range(0, len(input_data), batch_size):
            batch_input = [list(map(lambda x: input2id[x], sent.split())) for sent in input_data[i:i+batch_size]]
            batch_output = [list(map(lambda x: output2id[x], sent.split())) for sent in output_data[i:i+batch_size]]
            batch_output_length = [len(x) for x in batch_output]
            max_output_length = max(batch_output_length)
            batch_output = [x + [output2id["<PAD>"]] * (max_output_length - len(x)) for x in batch_output]
            _, batch_loss = sess.run([optimizer, loss], feed_dict={
                inputs: batch_input,
                outputs: batch_output,
                output_length: batch_output_length
            })
            epoch_loss += batch_loss
        print("Epoch:", epoch + 1, "Loss:", epoch_loss / len(input_data))

    # 测试模型
    test_input = "2030年6月1日"
    test_input = [input2id[x] for x in test_input.split()]
    test_input = np.array(test_input).reshape([1, -1])
    test_output = np.zeros([1, len(test_input) + 1], dtype=np.int32)
    test_output[0, 0] = output2id["<GO>"]
    for i in range(len(test_input)):
        test_input_length = np.array([len(test_input)]).reshape([1])
        test_logits = sess.run(decoder_logits, feed_dict={
            inputs: test_input,
            outputs: test_output,
            output_length: test_input_length
        })
        pred = np.argmax(test_logits, axis=2)
        test_output[0, i+1] = pred[0, i]
    test_output = [id2word_output[x] for x in test_output[0, :]]
    test_output = "".join(test_output[1:-1])
    print("%s -> %s" % (test_input[0], test_output))


AttributeError: ignored